## Note: mode1-taxi, mode2-FHV, mode3-shared FHV, mode4-PT, mode5-walking

In [17]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import time
import timeit
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV

## Data

In [6]:
acs = pd.read_csv('final_acs_transportation_choice.csv', index_col=0)
acs.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
0,3,177.762595,73.068204,1.150001,16668.822855,2001.196346
1,4,201.297419,120.885755,61.223127,13318.123341,6279.470358
2,5,49.601278,7.992997,0.306749,8847.276451,142.822526
3,6,164.390442,40.427858,2.346380,22929.859149,2485.976170
4,7,616.150103,162.371679,110.267353,138012.557541,11998.653324


In [7]:
mode_data = pd.read_csv('final_allMode_with_wage.csv', index_col=0).set_index('ODpair')
mode_data = mode_data.dropna()
mode_data['duration'] = mode_data.duration/60
mode_data.head(10)

,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,mode,nest
ODpair,,,,,,,,,,,,,,,,,
3-1,3,1,56.650000,100.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,1,1
3-1,3,1,55.922881,126.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,2,1
3-1,3,1,62.420000,120.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3,1
3-1,3,1,128.000000,5.500000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,4,2
3-1,3,1,569.083333,0.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,5,3
3-2,3,2,140.000000,8.250000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,4,2
3-2,3,2,353.800000,0.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,5,3
3-3,3,3,6.508943,16.647073,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,1,1
3-4,3,4,39.695000,64.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,2,1


In [32]:
acs[:6]

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
0,3,177.762595,73.068204,1.150001,16668.822855,2001.196346
1,4,201.297419,120.885755,61.223127,13318.123341,6279.470358
2,5,49.601278,7.992997,0.306749,8847.276451,142.822526
3,6,164.390442,40.427858,2.346380,22929.859149,2485.976170
4,7,616.150103,162.371679,110.267353,138012.557541,11998.653324
5,9,10.952647,3.098351,0.067727,6398.135346,524.745929


In [30]:
mode_data[:7000]

,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,mode,nest
ODpair,,,,,,,,,,,,,,,,,
3-1,3,1,56.650000,100.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,1,1
3-1,3,1,55.922881,126.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,2,1
3-1,3,1,62.420000,120.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3,1
3-1,3,1,128.000000,5.500000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,4,2
3-1,3,1,569.083333,0.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,5,3
3-2,3,2,140.000000,8.250000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,4,2
3-2,3,2,353.800000,0.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,5,3
3-3,3,3,6.508943,16.647073,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,1,1
3-4,3,4,39.695000,64.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,2,1


In [8]:
mode_data.shape[0]/5

56254.0

## Modeling

### Nested Logit Model

In [21]:
def denoSum(T, nestList, wage, Lambda, dataset):
    '''
    Calculate the denomenator for P(y=Nk)
    T: the dict that contains Tk for each Nest, in our case T={1:T1, 2:T2, 3:T3}; T1, T2, T3 defined by us 
    TotK: the total number of nest this model has, in our case TotK = 3
    '''
    deno = 0
    dictIVk = {}
    dictVj = {}
    for Nk in nestList: #k is the k Nest, in our case k=1,2,3
        Tk = T[Nk] #get the tao for nest k 
        subsetNk = dataset[dataset['nest'] == Nk]
        modes = list(subsetNk['mode'].unique()) #what modes contained in this nest k
    #     print(modes)
        sumIV = 0
        for j in (modes):   
            subset = dataset[dataset['mode'] == j]
            vj = Lambda * (float(int(wage)/124800) * float(subset['duration']) + float(subset['price']))
#             vj = utility(j, wage, Lambda, subsetNk)
            dictVj[j] = -vj
            sumIV += np.exp((1/Tk)*vj)  
    #     print(sumIV,Nk)
        IVk = (sumIV)
#         dictVj, IVk = InclusiveValue(Nk, Tk, wage, dictVj, Lambda, dataset)
        denok = np.exp(Tk*np.log(IVk))
        deno += denok
        dictIVk[Nk] = IVk
    return dictVj, dictIVk, deno


# def InclusiveValue(Nk, Tk, wage, dictVj, Lambda, dataset):
#     '''
#     Get the inclusive value for nest K
#     Nk:nest k
#     T: the dict that contains Tao(dissmilarity parameter) for each Nest. for example, Tk meeas Tao for nest K
#     '''
#     subsetNk = dataset[dataset['nest'] == Nk]
#     modes = list(subsetNk['mode'].unique()) #what modes contained in this nest k
# #     print(modes)
#     sumIV = 0
#     for j in (modes):   
#         vj = utility(j, wage, Lambda, subsetNk)
#         dictVj[j] = vj
#         sumIV += np.exp((1/Tk)*vj)  
# #     print(sumIV,Nk)
#     IVk = (sumIV)
# #     print(IVk,Nk)
#     return dictVj, IVk
    

# def utility(mode, wage, Lambda, dataset):
#     '''
#     Get the utility for mode j under specific OD pair
    
#     Arguments:
#         mode:
#         wage:
#         Lambda: parameter that trade-off different transportation mode
#         dataset:
        
#     Returns: 
#         -vj: Negative utility for the given 
#     '''
#     subset = dataset[dataset['mode'] == mode]
#     vj = Lambda * (float(int(wage)/124800) * float(subset['duration']) + float(subset['price'])) #525600: convert wage scale 'year' to 'minutes'
#     return -vj #assign negative utility?


def probability (j, Nk, T, dictVj, dictIVk, deno):  
    '''
    Calculate the probability for the specific mode j and Nest Nk
    '''
    Tk = T[Nk]
    IVk = dictIVk[Nk]
    vj = dictVj[j]
    pjk = np.exp((1/Tk)*vj)/(IVk) #pjk: P(y=j, y belong to Nk)
    pk = np.exp(Tk*np.log(IVk))/deno #pk: P(y belong to Nk)
    pj = pjk*pk #pj: P(y=j)
    return pj

### Apply to our case

In [22]:
def apply_model(T, nestList, wageList, Lambda, dataAll):
    '''
    apply to our case
    '''
    start = timeit.default_timer()

    ODpair_list = list(dataAll.index.unique())
#     print(len(ODpair_list))
    p = []
    for i in tqdm(ODpair_list,position=0): #identify OD pair  
        dataOD = dataAll[dataAll.index==i] 
        modeList = list(dataOD['mode'])
        nestList = list(dataOD['nest'])
        pop_OD = [] #store the results under each OD pair
        for wage in (wageList):            
            dictVj, dictIVk, deno = denoSum(T, set(nestList), wage, Lambda, dataOD)
            pop_mode = [] #store the results under each OD pair and each wage
            for i in (range(1,6)):
                if i in modeList: #not all modes appear in every OD pair
                    pop = dataOD[wage].mean() * probability(i, nestList[modeList.index(i)], T, dictVj, dictIVk, deno)
                    if np.isnan(pop) == True: #if predicted population is nan, replace it as 0, means no people choose
                        pop = 0
                
                    pop_mode.append(pop)
                else: 
                    pop_mode.append(0)
            pop_OD.append(pop_mode)
        pop_OD_sum = [sum(x) for x in zip(*pop_OD)] #sum the weighted population under each wage scenario
        p.append(pop_OD_sum)

    df = pd.DataFrame(p, columns=['P(mode1)', 'P(mode2)', 'P(mode3)', 'P(mode4)', 'P(mode5)']
                      , index=ODpair_list)
    
    stop = timeit.default_timer()
    timeslot = stop - start
    return df, timeslot

def compare_with_ground_truth(predictdf, truedf):
    '''
    compare our predicted transportation choice with ground truth
    
    The header of the datafrme after merge (named 'data_compare') should be like:
    taxi_zone | P(mode1)_x | P(mode2)_x | P(mode3)_x | P(mode4)_x | P(mode5)_x | P(mode1)_y | P(mode2)_y | P(mode3)_y | P(mode4)_y | P(mode5)_y
    '''
    import numpy as np
    # makesure predictdf and truedf have the same formats
    for col in predictdf.columns:
        predictdf[col] = predictdf[col].astype(float) 
    predictdf = predictdf.fillna(0)
    predictdf = predictdf.replace([np.inf, -np.inf], np.nan)
    predictdf = predictdf.dropna()
    predictdf['taxi_zone'] = predictdf.index.map(lambda x: x.split('-')[0]) #get origin taxi zone from each OD pair
    predictdf = predictdf.groupby('taxi_zone').sum().reset_index() #group the popuation by taxi zone
    predictdf['taxi_zone'] = predictdf['taxi_zone'].astype(int)
    truedf['taxi_zone'] = truedf['taxi_zone'].astype(int)
    
    data_compare = pd.merge(predictdf, truedf, left_on='taxi_zone', right_on = 'taxi_zone', how = 'left')
    data_compare = data_compare.dropna() 
    
    rmse = 0
    for i in range(1,6):
        rmsei = np.sqrt(sum((data_compare[data_compare.columns[i]] - data_compare[data_compare.columns[i+5]])**2))
    rmse += rmsei
    return data_compare, rmse

In [23]:
#choose ODpair 3-1 to test the algrithm whether bug-free:
T1 = 10
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} #Tao for each nest
Lambda = 2 
nestList = [1, 2, 3]
wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']
testdf = mode_data[mode_data.index.isin(list(mode_data.index[:10]))] #test OD pair 3-1,3-2,3-3,3-4

predict_choice_test, timeslot_test = apply_model(T, nestList, wagelist, Lambda, testdf)
results_test,rmse_test = compare_with_ground_truth(predict_choice_test, acs)
print('The time used to run the code:', timeslot_test)
print('The rmse of this model is:', rmse_test)
print()
print('The predict transportation choice is:')
predict_choice_test

100%|██████████| 4/4 [00:00<00:00, 32.67it/s]

The time used to run the code: 0.14676055999996152
The rmse of this model is: 2001.196345617838

The predict transportation choice is:


,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-1,1.904633e-17,1.059236e-19,3.275918e-19,1.951519e-116,2.471326e-119
3-2,0.000000e+00,0.000000e+00,0.000000e+00,4.947773e-17,1.381116e-13
3-3,1.012425e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3-4,0.000000e+00,7.593772e-09,1.159536e-08,1.999922e-62,3.901562e-60


In [35]:
np.logspace(-5,5,50)

array([1.00000000e-05, 1.59985872e-05, 2.55954792e-05, 4.09491506e-05,
       6.55128557e-05, 1.04811313e-04, 1.67683294e-04, 2.68269580e-04,
       4.29193426e-04, 6.86648845e-04, 1.09854114e-03, 1.75751062e-03,
       2.81176870e-03, 4.49843267e-03, 7.19685673e-03, 1.15139540e-02,
       1.84206997e-02, 2.94705170e-02, 4.71486636e-02, 7.54312006e-02,
       1.20679264e-01, 1.93069773e-01, 3.08884360e-01, 4.94171336e-01,
       7.90604321e-01, 1.26485522e+00, 2.02358965e+00, 3.23745754e+00,
       5.17947468e+00, 8.28642773e+00, 1.32571137e+01, 2.12095089e+01,
       3.39322177e+01, 5.42867544e+01, 8.68511374e+01, 1.38949549e+02,
       2.22299648e+02, 3.55648031e+02, 5.68986603e+02, 9.10298178e+02,
       1.45634848e+03, 2.32995181e+03, 3.72759372e+03, 5.96362332e+03,
       9.54095476e+03, 1.52641797e+04, 2.44205309e+04, 3.90693994e+04,
       6.25055193e+04, 1.00000000e+05])

In [33]:
#Run the model for the whole dataset
T1 = 0.05 #should set grid search for T1 
T2 = 1
T3 = 1
T = {1:T1, 2:T2, 3:T3} #Tao for each nest
Lambda = 0.45 #should set grid search for Lambda 
nestList = [1, 2, 3]
wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']

predict_transportation_choice, timeslot = apply_model(T, nestList, wagelist, Lambda, mode_data[:7000])
results,rmse = compare_with_ground_truth(mode_data[:7000], acs[:6])

100%|██████████| 1816/1816 [01:09<00:00, 28.01it/s]


In [ ]:
param_grid ={'Lambda':np.logspace(-5,5,50),'T[1]':np.logspace(-5,5,50)}

# dt=DecisionTreeClassifier(random_state=2019)
gr=GridSearchCV(dt,param_grid=param_grid,cv=None)
gr.fit(X_train,y_train)
print(gr.best_params_)
print(gr.score(X_test,y_test))

In [43]:
rmse_best = np.inf

In [51]:
list(zip(*[10,12]))

TypeError: zip argument #1 must support iteration

In [ ]:
for Tau in np.logspace(-5,5,50):
    for lamb in np.logspace(-5,5,50):
        T1 = Tau #should set grid search for T1 
        T2 = 1
        T3 = 1
        T = {1:T1, 2:T2, 3:T3} #Tao for each nest
        Lambda = lamb #should set grid search for Lambda 
        nestList = [1, 2, 3]
        wagelist = ['2500', '7500', '12500', '17500', '22500', '30000', '42500', '62500', '87500', '125000', '225000']

        predict_transportation_choice, timeslot = apply_model(T, nestList, wagelist, Lambda, mode_data[:7000])
        results,rmse = compare_with_ground_truth(mode_data[:7000], acs[:6])
        if rmse < rmse_best:
            predictions_best = predict_transportation_choice.copy()
            T1_best = T1
            Lambda_best = Lambda
            rmse_best = rmse
            results_best = results.copy()
        

 18%|█▊        | 335/1816 [00:13<01:04, 23.05it/s]

In [25]:
print('The time used to run the code:', timeslot)
print('The rmse of this model is:', rmse)
print()
print('The predict transportation choice is:')
predict_transportation_choice

The time used to run the code: 37.20573355800002
The rmse of this model is: 1258318.0134910252

The predict transportation choice is:


,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3-2,0.000000e+00,0.000000e+00,0.000000e+00,7.431783e-142,5.184974e-116
3-3,2.642425e-19,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3-4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3-5,0.000000e+00,0.000000e+00,0.000000e+00,4.047845e-148,2.732046e-143
3-6,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3-7,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3-8,0.000000e+00,0.000000e+00,0.000000e+00,1.256175e-90,1.964959e-73
3-9,0.000000e+00,3.169758e-49,9.478857e-49,4.845993e-319,7.901145e-311
3-10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
